In [ ]:
### verificar la instalación ###

In [1]:
import findspark
findspark.init()

In [12]:
import pyspark
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder.getOrCreate()

df = spark.sql("select 'spark' as hello ")

df.show

<bound method DataFrame.show of DataFrame[hello: string]>

## Probando la sesión de Spark

In [14]:
df = spark.createDataFrame([{"Hola": "Mundo"} for x in range(10)])

df.show(10, False)

+-----+
|Hola |
+-----+
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
+-----+



# Ejercicio 2
* Cree una sesión de Spark con nombre Cap2 y asegúrese de que emplea todos los cores disponibles para ejecutar en su ambiente de trabajo.
* Cree dos RDD vacíos, uno de ellos no debe contener particiones y el otro debe tener 5 particiones. Utilice vías diferentes para crear cada RDD.
* Cree un RDD que contenga los números primos que hay entre 1 y 20.
* Cree un nuevo RDD a partir del RDD creado en el ejercicio anterior el cuál solo contenga los números primos mayores a 10.
* Descargue el archivo de texto adjunto a esta lección como recurso y guárdelo en una carpeta llamada data en el ambiente de trabajo de Colab.
* Cree un RDD a partir de este archivo de texto en donde todo el documento esté contenido en un solo registro. ¿Cómo podría saber la dirección donde está guardado el archivo de texto a partir del RDD creado?
* Si necesitara crear un RDD a partir del archivo de texto cargado previamente en donde cada línea del archivo fuera un registro del RDD, ¿cómo lo haría?

In [15]:
sc = spark.sparkContext
rdd1 = sc.emptyRDD()

In [16]:
rdd2 = sc.parallelize([],5)

In [17]:
rdd1.getNumPartitions()

0

In [18]:
rdd_primo = sc.parallelize([2, 3, 5, 7, 11, 13, 17, 19])

In [19]:
rdd_primo.filter(lambda x: x > 10).collect()

[11, 13, 17, 19]

In [15]:
rdd_texto = sc.wholeTextFiles("file:///C:/el_valor_del_big_data.txt")

In [16]:
rdd_texto.collect()

[('file:/C:/Users/diana.cardozo_blueta/Downloads/Recursos_cursoSpark/el_valor_del_big_data.txt',
  'El valor y la realidad de big data\r\nEn los últimos años, han surgido otras "dos V": valor y veracidad. Los datos poseen un valor intrínseco. Sin embargo, no tienen ninguna utilidad hasta que dicho valor se descubre. Resulta igualmente importante: ¿cuál es la veracidad de sus datos y cuánto puede confiar en ellos?\r\n\r\nHoy en día, el big data se ha convertido en un activo crucial. Piense en algunas de las mayores empresas tecnológicas del mundo. Gran parte del valor que ofrecen procede de sus datos, que analizan constantemente para generar una mayor eficiencia y desarrollar nuevos productos.\r\n\r\nAvances tecnológicos recientes han reducido exponencialmente el coste del almacenamiento y la computación de datos, haciendo que almacenar datos resulte más fácil y barato que nunca. Actualmente, con un mayor volumen de big data más barato y accesible, puede tomar decisiones empresariales m

## Crear un RDD de una colección

In [16]:
languajes = sc.parallelize(['Python', 'R', 'C', 'Scala','SQL'])

In [17]:
languajes.collect()

['Python', 'R', 'C', 'Scala', 'SQL']

In [19]:
rdd_may = languajes.map(lambda x: x.upper())
rdd_min = languajes.map(lambda x: x.lower())

In [21]:
rdd_may.collect()

['PYTHON', 'R', 'C', 'SCALA', 'SQL']

In [22]:
rdd_min.collect()

['python', 'r', 'c', 'scala', 'sql']

In [24]:
rdd_r = languajes.filter(lambda x: x.startswith('R'))

In [20]:
pares = sc.parallelize([20, 22, 24, 26, 28, 30])
pares.collect()

[20, 22, 24, 26, 28, 30]

## transformaciones 

* las transformaciones son de naturaleza "perezosa" y no se ejecutan hasta que se ejecute una accion sobre estas 

## Map

In [21]:
import math
sqrt = pares.map(lambda x: math.sqrt(x))
sqrt.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

## flatMap

In [24]:
lista = pares.flatMap(lambda x: (x, math.sqrt(x))).collect()
print(lista)

[20, 4.47213595499958, 22, 4.69041575982343, 24, 4.898979485566356, 26, 5.0990195135927845, 28, 5.291502622129181, 30, 5.477225575051661]


## Filtro

In [32]:
even_rdd = pares.filter(lambda x: x % 3 == 0)
even_rdd.collect()

[24, 30]

## flatMap

### distinct

* Esto devolvera elementos diatintos a un RDD

In [35]:
rdd1 = sc.parallelize([10, 11, 10, 11, 12, 11])
dist_rdd = rdd1.distinct()
dist_rdd.collect()

[10, 11, 12]

### reduceByKey
* Esta función reduce los pares de valores clave en función de las claves y una función dterminada dentro de reduceBy Key

In [44]:
pairs = [("a", 5), ("b", 7), ("c", 2), ("a", 3), ("b",1), ("c", 4)]
pair_rdd = sc.parallelize(pairs)

output = pair_rdd.reduceByKey(lambda x, y: x + y)

result = output.collect()
print(*result, sep='\n')

('c', 6)
('a', 8)
('b', 8)


### groupByKey
* Esta funcion puede operar en un par(clave, valor) RDD pero esto solo agrupará los valores basadas en las claves.

In [46]:
grp_out = pair_rdd.groupByKey()
grp_out.collect()

[('c', <pyspark.resultiterable.ResultIterable at 0x16a87831450>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x16a877ac050>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x16a867a5f90>)]

### sortByKey
* Clasificación en orden ascendente

In [47]:
pairs = [("a", 5), ("d", 7), ("c", 2), ("b", 3)]
raw_rdd = sc.parallelize(pairs)
sortkey_rdd = raw_rdd.sortByKey()
result = sortkey_rdd.collect()
print(*result, sep='\n')

('a', 5)
('b', 3)
('c', 2)
('d', 7)


## Ordenar por
* otro criterio

In [22]:
# Cramos el rdd
pairs = [("a", 5, 10), ("d", 7, 12), ("c", 2, 11), ("b", 3, 9)]
raw_rdd = sc.parallelize(pairs)

# Ordenación basada en le tercer elemento de la tupla
sort_out = raw_rdd.sortBy(lambda x: x[2])
result = sort_out.collect()
print(*result, sep='\n')

('b', 3, 9)
('a', 5, 10)
('c', 2, 11)
('d', 7, 12)
